In [2]:
from statsmodels.tsa.vector_ar.var_model import VAR
import pandas as pd

In [3]:
import matplotlib.pyplot as plt

In [4]:
df = pd.read_parquet('subbrand_grouped.parquet')

In [5]:
df['DATE_FIELD'] = pd.to_datetime(df['DATE_FIELD'])
df['YEAR'] = df['DATE_FIELD'].dt.year
df['MONTH'] = df['DATE_FIELD'].dt.month
df['DAY'] = df['DATE_FIELD'].dt.day

In [6]:
min_sales_volume = 1000  
min_sales_years = 2     
max_zero_sales_percentage = 75

In [7]:
relevant_subbrands = []

In [8]:
for subbrand, subbrand_data in df.groupby('ITM_SBRND_CD'):
    total_sales_volume = subbrand_data['SALES'].sum()
    years_with_sales = subbrand_data['YEAR'].nunique()
    zero_sales_percentage = (subbrand_data['SALES'] == 0).mean() * 100
    
    if (total_sales_volume >= min_sales_volume and
        years_with_sales >= min_sales_years and
        zero_sales_percentage < max_zero_sales_percentage):
        relevant_subbrands.append(subbrand)


In [9]:
df = df[df['ITM_SBRND_CD'].isin(relevant_subbrands)]

In [10]:
X = df[['ITM_SBRND_CD', 'YEAR', 'MONTH', 'DAY']]
y = df['SALES']

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
models_var = {}
predictions_var = {}

In [18]:
for group_name, group_data in df.groupby('ITM_SBRND_CD'):
    # Skip groups with fewer than 2 samples
    if len(group_data) < 2:
        continue

    
    group_data = group_data[['DATE_FIELD', 'SALES' , 'ITM_SBRND_CD']].set_index('DATE_FIELD')

    train_size = int(0.8 * len(group_data))
    train, test = group_data.iloc[:train_size], group_data.iloc[train_size:]

    model_var = VAR(train)
    model_var_fit = model_var.fit()

    lag_order = model_var_fit.k_ar
    pred = model_var_fit.forecast(train.values[-lag_order:], len(test))

    models_var[group_name] = model_var_fit
    predictions_var[group_name] = {'Actual': test['SALES'].values, 'Predicted': pred[:, 0]}

    plt.figure(figsize=(12, 6))
    plt.plot(test.index, test['SALES'], label='Actual')
    plt.plot(test.index, pred[:, 0], label='Predicted')
    plt.title(f'SUBBRND_CODE: {group_name}')
    plt.xlabel('Date')
    plt.ylabel('Sales')
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()

/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ValueError: x contains one or more constant columns. Column(s) 1 are constant. Adding a constant with trend='c' is not allowed.